In [1]:
import pandas as pd
import glob
import os
import re

import re
import pandas as pd
import os
import pytz
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import warnings
from concurrent.futures import ThreadPoolExecutor
import shutil

##### Date range correction
* Only dates after July 2023 will be taken

In [3]:
# Hourly average time shiftinhg
dirs = ["merged/Indoor", "merged/Outdoor"]
for dir in dirs:
    print('='*50)
    print(f"Processing {dir}")
    csv_files = [f for f in os.listdir(dir) if f.endswith('.csv')]
    for file in csv_files:
        filedir = os.path.join(dir, file)
        df = pd.read_csv(filedir, parse_dates=['UTCDateTime'])
        df["BDDateTime"] = df['UTCDateTime'] + pd.Timedelta(hours=6)
        df = df[df['BDDateTime'] >= pd.to_datetime("2023-07-01", format = "%Y-%m-%d")]
        df = df.drop('UTCDateTime', axis=1)
        df.to_csv(filedir, index=False)
        print("Saved:", filedir)


Processing merged/Indoor
Saved: merged/Indoor/Mohammadpur.csv
Saved: merged/Indoor/Dhakeshwari.csv
Saved: merged/Indoor/Mohakhali.csv


/tmp/ipykernel_4775/3236335387.py:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filedir, parse_dates=['UTCDateTime'])


Saved: merged/Indoor/Banasree.csv
Saved: merged/Indoor/Nakhalpara.csv
Saved: merged/Indoor/Khilgawn.csv
Processing merged/Outdoor
Saved: merged/Outdoor/Mohammadpur.csv
Saved: merged/Outdoor/Dhakeshwari.csv
Saved: merged/Outdoor/Mohakhali.csv
Saved: merged/Outdoor/Banasree.csv
Saved: merged/Outdoor/Nakhalpara.csv


/tmp/ipykernel_4775/3236335387.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filedir, parse_dates=['UTCDateTime'])


Saved: merged/Outdoor/Khilgawn.csv


##### Combinig 2 channels 
* either on average, or one of the channels

In [ ]:
# ## merging channel

# conditions =   {'Mohakhali': {'indoor': {'atm': 'average', 'cf': 'average'},
#                                     'outdoor': {'atm': 'average', 'cf': 'average'}},

#                 'Mohammadpur': {'indoor': {'atm': 'average', 'cf': 'average'},
#                                     'outdoor': {'atm': 'b', 'cf': 'b'}},

#                 'Banasree': {'indoor': {'atm': 'b', 'cf': 'b'},
#                                     'outdoor': {'atm': 'average', 'cf': 'average'}},

#                 'Nakhalpara': {'indoor': {'atm': 'average', 'cf': 'average'},
#                                     'outdoor': {'atm': 'average', 'cf': 'average'}},

#                 'Dhakeshwari': {'indoor': {'atm': 'average', 'cf': 'average'},
#                                     'outdoor': {'atm': 'average', 'cf': 'average'}},

#                 'Khilgawn Home': {'indoor': {'atm': 'a', 'cf': 'a'},
#                                     'outdoor': {'atm': 'a', 'cf': 'a'}}}



# ## average calculation
# dir = "merged"
# subdirs = ['indoor', 'outdoor']
# for subdir in tqdm(subdirs):
#     print(f'processing {subdir}')
#     files = [file for file in os.listdir(os.path.join(dir, subdir)) if file.endswith('.csv')]
#     for file in files:
#         home_name = file.split('.')[0]
#         df = pd.read_csv(os.path.join(dir, subdir, file))
#         cond = conditions[home_name][subdir]
#         # 'pm1_0_cf_1', 'pm2_5_cf_1', 'pm10_0_cf_1', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm', 'pm1_0_cf_1_b', 'pm2_5_cf_1_b', 'pm10_0_cf_1_b', 'pm1_0_atm_b', 'pm2_5_atm_b', 'pm10_0_atm_b']
#         datapoints = ['pm1_0_cf_1', 'pm2_5_cf_1', 'pm10_0_cf_1', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm']
#         for dp in datapoints:
#             if "cf" in dp:
#                 channel = 'cf'
#             elif 'atm' in dp:
#                 channel = 'atm'
#             else:
#                 raise ValueError(f"Unknown data point channel in {dp}")
#             cond = conditions[home_name][subdir][channel]
#             if cond == 'average':
#                 df[dp + "_comb"] = (df[dp] + df[dp + "_b"])/2
#             if cond == 'b':
#                 df[dp + "_comb"] = df[dp + "_b"]
#             if cond == 'a':
#                 df[dp + "_comb"] = df[dp]

#         df = df.drop(columns=datapoints + [dp + "_b" for dp in datapoints])

#         target = os.path.join("combined", subdir, f'{home_name}.csv')
#         df.to_csv(target, index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

processing indoor
processing outdoor


#### Hourly average

In [4]:
def hourly_averages(df, datas, time='UTCDateTime', calibrate = False, subdir = None, merge = None):
    results = {"time" : []}
    for col in datas:
        results[col] = []
    for col in datas:
        results[col+"count"] = []
    

    # Ensure the DataFrame has the correct time and data columns
    df = df[datas + [time]].copy()
    # df = df.dropna()  # Remove rows with NaN in time or data columns
    df[time] = pd.to_datetime(df[time])  # Ensure time column is datetime
    df = df.sort_values(by=time)  # Sort by time column

    # Generate specific hourly times for calculation (00:30, 01:30, ..., 23:30)
    start_time = df[time].min().replace(minute=30, second=0, microsecond=0)
    end_time = df[time].max().replace(minute=30, second=0, microsecond=0)
    hourly_times = pd.date_range(start=start_time, end=end_time, freq="1h")

    for target_time in tqdm(hourly_times, desc=f"Taking hourly averages for {subdir} ({merge})"):
        # Append the results
        results["time"].append(target_time)
        # Define the 1-hour window (30 minutes before and after)
        start_window = target_time - pd.Timedelta(minutes=30)
        end_window = target_time + pd.Timedelta(minutes=30)
        # Filter data within the window
        window_data_all = df[(df[time] >= start_window) & (df[time] <= end_window)][datas]
        for data in datas:
            # Count the number of data points in the window
            window_data = window_data_all[data].dropna()
            count = len(window_data)
            # Compute the average if the number of points is >= 10
            if count >= 10:
                avg = window_data.mean()
                
            else:
                avg = np.nan  # Fill with NaN if insufficient data
            
            results[data].append(avg)
            results[data+'count'].append(count)

    # Convert results to a DataFrame for convenience
    results_df = pd.DataFrame(results)
    return results_df

In [11]:
# Hourly average
dirs = [
    # "merged/Indoor", 
    "merged/Outdoor"]
for dir in dirs:
    print('='*50)
    print(f"Processing {dir}")
    # files = [file for file in os.listdir(dir) if file.endswith('.csv')]
    files = ['Khilgawn.csv']
    for file in files:
        if False and os.path.exists(os.path.join(dir, subdir, f'Hourly_average_{type}.csv')):
            print("file found. skipping")
            continue
        
        filedir = os.path.join(dir, file)
        df = pd.read_csv(filedir)
        df['BDDateTime'] = pd.to_datetime(df['BDDateTime'])
        hourly_avg = hourly_averages(df, datas=['current_temp_f', 'current_humidity', 'pm2_5_atm', 'p_0_3_um', 'p_0_5_um', 'p_1_0_um', 'p_2_5_um', 'p_5_0_um', 'p_10_0_um'], time='BDDateTime')
        target = os.path.join('hourly_averages', dir.split('/')[-1], file)
        hourly_avg.to_csv(target, index=False)
        print(f"saved : {target}")

Processing merged/Outdoor


Taking hourly averages for None (None):   0%|          | 0/10410 [00:00<?, ?it/s]

saved : hourly_averages\Outdoor\Khilgawn.csv


In [2]:
pd.read_csv('merged/Outdoor/Khilgawn.csv')

C:\Users\Dipankar Mitra\AppData\Local\Temp\ipykernel_26036\3729885224.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv('merged/Outdoor/Khilgawn.csv')


,current_temp_f,current_humidity,pm2_5_atm,p_0_3_um,p_0_5_um,p_1_0_um,p_2_5_um,p_5_0_um,p_10_0_um,BDDateTime
0,89.0,68.0,64.32,10085.66,2907.77,723.50,67.91,19.73,4.91,2023-09-02 06:47:21
1,89.0,68.0,64.82,10246.69,2934.55,734.15,70.44,23.45,5.53,2023-09-02 06:49:21
2,89.0,68.0,62.25,9896.77,2835.21,696.34,75.96,26.42,5.66,2023-09-02 06:51:21
3,89.0,68.0,61.05,9573.16,2756.68,691.21,69.04,26.46,6.32,2023-09-02 06:53:21
4,89.0,68.0,58.73,9342.11,2676.05,661.75,68.25,26.96,3.46,2023-09-02 06:55:21
...,...,...,...,...,...,...,...,...,...,...
298071,88.0,70,34.34,4243.30,1214.19,265.96,29.77,14.45,4.98,2023-08-20 05:50:24
298072,88.0,70,34.17,4216.44,1188.56,272.85,34.96,16.22,5.63,2023-08-20 05:52:24
298073,88.0,70,33.85,4144.06,1188.81,272.00,30.93,15.74,3.48,2023-08-20 05:54:24
298074,88.0,70,34.08,4196.43,1195.17,266.34,33.21,16.68,2.98,2023-08-20 05:56:24


### Code written by

Rivan Chandra Roy
<br>
Bangladesh University of Engineering and Technology